# Model 1 - Baseline

## Import the Preprocessed Data

In [101]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

trainFeatures = pd.read_csv("../../data/preprocessed/pipeline1/trainFeatures.csv")
trainLabels = pd.read_csv("../../data/preprocessed/pipeline1/trainLabels.csv")
testData = pd.read_csv("../../data/preprocessed/pipeline1/test.csv")

In [102]:
trainFeatures.head()

,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,1.000000,0.2750,0.125,0.0,0.014151,0,0,1,0,1
1,0.333333,0.4750,0.125,0.0,0.139136,1,0,0,1,0
2,1.000000,0.3250,0.000,0.0,0.015469,0,0,1,1,0
3,0.333333,0.4375,0.125,0.0,0.103644,0,0,1,1,0
4,1.000000,0.4375,0.000,0.0,0.015713,0,0,1,0,1


In [103]:
trainLabels.head()

,Survived
0,0
1,1
2,1
3,1
4,0


## Create Validation Dataset (20% of Training Data)

In [104]:
splittingPoint = int(len(trainFeatures) * 0.8)

validationFeatures = trainFeatures[splittingPoint:]
validationLabels = trainLabels[splittingPoint:]

trainFeatures = trainFeatures[:splittingPoint]
trainLabels = trainLabels[:splittingPoint]

## Create TensorFlow Datasets

In [105]:
trainDataset = tf.data.Dataset.from_tensor_slices((trainFeatures, trainLabels)).batch(32).prefetch(tf.data.AUTOTUNE)
validationDataset = tf.data.Dataset.from_tensor_slices((validationFeatures, validationLabels)).batch(32).prefetch(tf.data.AUTOTUNE)
testDataset = tf.data.Dataset.from_tensor_slices(testData).batch(32).prefetch(tf.data.AUTOTUNE)

## Create the Baseline Model and save the best epoch

In [106]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=64, activation="relu"),
    tf.keras.layers.Dense(units=64, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)

history = model.fit(
    x=trainDataset,
    validation_data=validationDataset,
    epochs=50,
    callbacks=[tf.keras.callbacks.ModelCheckpoint("../trainedmodels/saved models/model1", verbose=0)],
    verbose=0
)